In [19]:
import numpy as np
import random
import pandas as pd
from tqdm import tqdm
from collections import deque
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam

from importlib import reload
from utilities import *
import agent
reload(agent)

<module 'agent' from '/Users/nagajas/Desktop/rl_trader_AI/agent.py'>

In [20]:
stocks = ['AAPL', 'GOOGL', 'AMZN', 'ADBE']

In [21]:
episode_count = 10
true, diff = allStates(1000)
split = int(0.9*len(true[0]))

true_train, true_test = true[:split], true[:split]
diff_train, diff_test = diff[:split], diff[:split]

l = len(diff_train[0]) - 1

Model found for  AAPL
Model found for  GOOGL
Model found for  AMZN
Model found for  ADBE
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


In [22]:
p_agent = agent.DQNAgent(4)

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 4, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4, 64)          │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 4, 32)          │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4, 3)           │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,307 (9.01 KB)

 Trainable params: 2,307 (9.01 KB)

 Non-trainable params: 0 (0.00 B)

### $\textbf{Training Agent}$

In [26]:
for e in range(1,episode_count + 1):
    
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getstate(diff_train, 0)
    total_profit = 0
    p_agent.inventory = OrderedDict()
    p_agent.replay_buffer.clear()
    for stock in stocks:
        p_agent.inventory[stock] = []

    for t in tqdm(range(l)):
        actions = p_agent.epsilon_greedy(state)
        # sit
        next_state = getstate(diff_train, t+1)
        reward = np.zeros((4,1))
        
        for i in range(len(actions)):
            action = actions[i]
            stock = stocks[i]
            
            if action == 1: # buy
                p_agent.inventory[stock].append(true_train[i,t])
                
            elif action == 2 and len(p_agent.inventory[stock]) > 0: # sell
                bought_price = p_agent.inventory[stock].pop(0)
                reward[i] = np.max(true_train[i,t] - bought_price, 0)
                
                total_profit += true_train[i,t] - bought_price
        # print(reward)
        done = True if t == l - 1 else False
        p_agent.store_transition(state, actions, reward, next_state, done)
        state = next_state

        if len(p_agent.replay_buffer) > 32:
            p_agent.exp_replay(32)
            p_agent.replay_buffer.clear()        
        # print("t:::" , t)

Episode 1/10


100%|██████████| 926/926 [03:14<00:00,  4.75it/s]


Episode 2/10


100%|██████████| 926/926 [03:23<00:00,  4.56it/s]


Episode 3/10


...

100%|██████████| 926/926 [03:55<00:00,  3.93it/s]


Episode 9/10


100%|██████████| 926/926 [04:04<00:00,  3.79it/s]


Episode 10/10


100%|██████████| 926/926 [03:33<00:00,  4.34it/s]


### $\textbf{Evaluating Agent}$

In [31]:
state = getstate(diff_test, 0)
total_profit = 0
p_agent.inventory = OrderedDict()

for stock in stocks:
    p_agent.inventory[stock] = []

for t in range(len(diff_test[0])-1):
    actions = p_agent.epsilon_greedy(state)
    # sit
    next_state = getstate(diff_test, t+1)
    reward = np.zeros((4,1))
    
    print("Day: ", t+1)
    for i in range(len(actions)):
        action = actions[i]
        stock = stocks[i]
        
        if action == 0 and len(p_agent.inventory[stock]) > 0:
            print("Stock: ", stock, "Action: Hold: $", true_test[i,t])
        
        elif action == 1: # buy
            p_agent.inventory[stock].append(true_test[i,t])
            print("Stock: ", stock, "Action: Buy: $", true_test[i,t])
            
        elif action == 2 and len(p_agent.inventory[stock]) > 0: # sell
            bought_price = p_agent.inventory[stock].pop(0)
            reward[i] = np.max(true_test[i,t] - bought_price, 0)
            # print(reward)
            total_profit += true_test[i,t] - bought_price
            print("Stock: ",stock,"Action: Sell: $", (true_test[i,t]), " | Profit: $",(true_test[i,t] - bought_price))

    done = True if t == l - 1 else False
    p_agent.store_transition(state, actions, reward, next_state, done)
    state = next_state
    if done:
            print("-----------------------------------")
            print(" Total Profit: " ,total_profit)
            print("-----------------------------------")

Day:  1
Day:  2
Stock:  ADBE Action: Buy: $ 430.309998
Day:  3
Stock:  ADBE Action: Sell: $ 437.100006  | Profit: $ 6.790008
Day:  4
Day:  5
Stock:  AMZN Action: Buy: $ 151.676498
Day:  6
Stock:  AMZN Action: Hold: $ 152.593994
Day:  7
Stock:  AMZN Action: Buy: $ 158.233994
Day:  8
Stock:  AMZN Action: Hold: $ 155.594498
Day:  9
Stock:  GOOGL Action: Buy: $ 73.665001
Stock:  AMZN Action: Buy: $ 156.941498
Day:  10
Stock:  GOOGL Action: Sell: $ 73.954498  | Profit: $ 0.28949699999999723
Stock:  AMZN Action: Sell: $ 160.251495  | Profit: $ 8.574996999999996
Day:  11
Stock:  AAPL Action: Buy: $ 111.389778
Stock:  AMZN Action: Sell: $ 161.25  | Profit: $ 3.0160060000000044
Day:  12
Stock:  AAPL Action: Sell: $ 108.857254  | Profit: $ -2.5325240000000093
Stock:  AMZN Action: Sell: $ 158.373001  | Profit: $ 1.4315029999999922
Day:  13
Stock:  GOOGL Action: Buy: $ 74.841003
Stock:  AMZN Action: Buy: $ 157.408005
Day:  14
Stock:  AAPL Action: Buy: $ 107.155022
Stock:  GOOGL Action: Hold: $ 74.